### Purpose of This Notebook

This notebook will be used to fetch the climate at a glance data from the NOAA website. The data will be saved as csvs for easier access later.

In [1]:
import pandas as pd
import pandas.tseries
import numpy as np
import seaborn as sns
import requests
import json

In [2]:
def noaa_resp_to_ts(URL):
    """
    Accepts a link for JSON output from the publicly available NOAA climate at a glance data.
    Returns a timeseries  of this data.
    """
    response = requests.get(URL)
    
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        # Whoops it wasn't a 200
        return "Error: " + str(e)
    
    data = response.json()
    dates = list(data['data'].keys())
    dates = [date[:4] + '-' + date[4:] for date in dates]
    dates = pd.to_datetime(dates, format='%Y-%m')
    series = [v['value'] for v in data['data'].values()]
    ts = pd.Series(series, index=dates)
    ts = ts.astype(float)
    
    return ts

In [3]:
#Lists of metrics and urls for querying
metrics = ['tavg', 'tmax', 'tmin', 'pcp', 'cdd', 'hdd', 'pdsi', 'phdi', 'pmdi']
url_dict = {metric: [f'https://www.ncdc.noaa.gov/cag/statewide/time-series/{i}' + 
                     f'-{metric}-all-1-1940-2019.json?base_prd=true&begbaseyear=1940&endbaseyear=1940'
                     for i in range(1,49)] for metric in metrics}

In [ ]:
#Get all of the timeseries for each metric and each state
all_tseries = {metric: [noaa_resp_to_ts(url) for url in url_dict[metric]]
               for metric in metrics}

In [ ]:
#Combine timeseries into dataframes by metric
all_dfs = {metric: pd.DataFrame(all_tseries[metric]).T for metric in metrics}

In [ ]:
#Set column names for states and rename Date index
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH",
          "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA",
          "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA",
          "WV", "WI", "WY"]

for metric in metrics:
    all_dfs[metric].columns = states
    all_dfs[metric].index.rename('Date', inplace=True)
    all_dfs[metric].to_csv('cag_csvs/' + metric + '.csv')